In [1]:
from dotenv import load_dotenv
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")
sentiment_url = os.getenv("SENTIMENT_URL")

NameError: name 'os' is not defined

In [7]:
from langchain import PromptTemplate
from langchain.llms import OpenAI
import json

classify_intent_extract_entities_template = PromptTemplate(
    input_variables=["user_input", "intents", "entities"],
    template="""
    You are an intent and entity classification model.
    Given this list of intents: {intents},
    this list of entities {entities},
    and this user input {user_input}, 
    predict the intent and entities of the user input.

    Give the confidence level of each of your predictions. 
    Choose intents from the list of intents and only from the list of intents.
    Choose entities from the list of entities and only from the list of entities.
    Intent name creation is not allowed.
    Entity name creation is not allowed. If the list of entities 
    is empty, you can't predict any entities. Just predict the intent.
    You have to be really sure about the intent you are predicting, otherwise
    you can respond with the following JSON object :
        "intent"= "Redirigir_BaseDeConocimiento",
        "confidence"= 0.7
 

    Entities are always in the form of a list of dictionaries with this structure :
    "text"= extracted text from user input,"value"= value extracted, "name"= name of the entity, "confidence"= confidence level of the prediction

    Intent is always in the form of a dictionary with this structure=
    "name"= name of the intent, "confidence"= confidence level of the prediction

    Only respond with a valid JSON object and nothing else.
    Your response must have this structure:
        "intent"
        "entities"
    """,
)


async def classify_intent_extract_entities(user_input, intents, entities):

    llm = OpenAI(temperature=0.0)
    prompt = classify_intent_extract_entities_template.format(
        user_input=user_input,
        intents=intents,
        entities=entities,
    )
    response = llm(prompt)
    clean = "{" + response.split("{", 1)[1]
    clean = clean.replace("'", '"')
    print(clean)
    response = json.loads(clean)
    return response


In [8]:
user_input = "quiero adoptar un perro"

In [9]:
intents = [{"name":"Pregunta_Renovacion_Licencia","description":"El usuario no sabe si debe realizar la renovación de su licencia o no"},{"name":"Informacion_MesMujer","description":"Datos e información referida al mes de la mujer"},{"name":"Informacion_Robo_Licencia","description":"Información de los pasos a seguir cuando se perdió o te robaron la licencia"},{"name":"Informacion_LicenciaConducir","description":"Datos e información referida a la obtención y renovación de la licencia de conducir."},{"name":"Informacion_Infracciones","description":"Datos e información referida a infracciones de tránsito de motos, autos, y colectivos"},{"name":"Informacion_CorteLuz","description":"Datos e información referida a los cortes de luz en capital federal"},{"name":"Iniciar_Otorgamiento_Licencia","description":"Se desea realizar el trámite para sacar o renovar una licencia"},{"name":"jardin_botanico","description":"jardin_botanico"},{"name":"Informacion_ViolenciaGenero","description":"Datos e información referida a la violencia de género. Solicitudes de denuncias de violencia de género"}]

In [10]:
entities = [{"name":"patente","description":"la patente del auto con el formato 2 letras, 3 números y 2 letras. por ejemplo aa123aa"},{"name":"Documento","description":"Numero de identificacion nacional de una persona fisica, tambien conocido como DNI. Debe ser un número"},{"name":"Edad","description":"Edad de la persona"},{"name":"sexo","description":"Condición orgánica que distingue a los hombres y mujeres. Indicado como M para masculino, F para femenino y X para no binarie"}]

In [11]:
personality = "cool"

In [12]:
classified = await classify_intent_extract_entities(user_input, intents, entities)

{
        "intent": {
            "name": "Redirigir_BaseDeConocimiento",
            "confidence": 0.7
        },
        "entities": []
    }
}


JSONDecodeError: Extra data: line 8 column 1 (char 144)